# Limpeza dos Dados de Presença de Clientes de uma Barbearia

## Introdução
Este notebook trata os dados de presença de clientes de uma barbearia. Foi necessário fazer tratamento de exceção devido a muitos erros de digitação e falta de padronização no dataset original e, a partir desse tratamento, a correção manual desses erros foi facilitada.

## Importação das Bibliotecas

In [1]:
import pandas as pd
import numpy as np

## Funções de Conversão de Data

### Função `convert_date`

Converte datas de D/M/YY para DD/MM/YYYY.

In [2]:
def convert_date(date):
    day, month, year = date.split('/')
    
    formatted_day = "0" + day if len(day) < 2 else day
    formatted_month = "0" + month if len(month) < 2 else month
    formatted_year = "20" + year if len(year) < 4 else year
    
    formatted_date = f"{formatted_day}/{formatted_month}/{formatted_year}"
    return pd.to_datetime(formatted_date, format='%d/%m/%Y')

### Função `parse_presenca`

Processa a string de presença, separa e converte as datas.

In [3]:
def parse_presenca(dates):
    if pd.isna(dates):
        return []
    dates_list = dates.split(sep='-')
    dates_list = [convert_date(date.strip()) for date in dates_list if date != '']
    
    return dates_list

## Leitura do Dataset Original

In [4]:
raw_df = pd.read_csv('presenca_clientes.csv')
raw_df.head()

,data,id,presenca
0,28/7/2016,1,27/10/16-20/12/16-03/03/17-26/04/17-04/07/17-0...
1,28/7/2016,2,04/08/16 - 10/08/16 - 01/09/16-15/09/16-23/09/...
2,28/7/2016,3,10/08/16 -17/08/16-25/08/16-15/09/16-13/10/16-...
3,28/7/2016,4,13/09/16-05/10/16-01/11/16-15/03/17-
4,28/7/2016,5,09/08/16-


## Processamento dos Dados

Itera sobre cada linha, aplicando conversão e tratamento de exceções.

In [5]:
cleaned_df = pd.DataFrame(columns=['id', 'data', 'primeira_visita'])
next_index = 0

for _, row in raw_df.iterrows():
    id = row['id']
    
    try:
        first_date = convert_date(row['data'])
        dates = parse_presenca(row['presenca'])
    except Exception as e:
        print(f"Couldn't convert dates on line with id {id}: {e}")
        continue
    
    cleaned_df.loc[next_index] = {'id': id, 'data': first_date, 'primeira_visita': first_date}
    next_index += 1
    
    for date in dates:
        cleaned_df.loc[next_index] = {'id': id, 'data': date, 'primeira_visita': first_date}
        next_index += 1

In [6]:
cleaned_df.head(10)

,id,data,primeira_visita
0,1,2016-07-28,2016-07-28
1,1,2016-10-27,2016-07-28
2,1,2016-12-20,2016-07-28
3,1,2017-03-03,2016-07-28
4,1,2017-04-26,2016-07-28
5,1,2017-07-04,2016-07-28
6,1,2017-09-05,2016-07-28
7,1,2017-10-24,2016-07-28
8,1,2017-12-06,2016-07-28
9,1,2018-06-13,2016-07-28


In [7]:
cleaned_df.to_csv('presenca_clientes_limpo.csv', index=False)